Шаг 7. Построение рекомендательной системы
Сформировать датасет с признаками покупатель, товар, количество купленных товаров
Выполнить коллаборативную фильтрацию по схожести покупателей
Вывести в качестве рекомендаций список наименований товаров для пользователя с ИД = 6510

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scipy.sparse import csr_matrix

In [10]:
df_rating = pd.read_csv("https://github.com/OlesiaAngel/DataAnalitics/blob/main/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5/stat/transactions.csv?raw=true", index_col=0)

df_custom = pd.read_csv("https://github.com/OlesiaAngel/DataAnalitics/blob/main/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5/stat/customers.csv?raw=true", index_col=0)

df_item = pd.read_csv("https://github.com/OlesiaAngel/DataAnalitics/blob/main/%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5/stat/product.csv?raw=true", index_col=0)

In [11]:
df = df_rating[["customer_id","article_id"]].value_counts().reset_index() \
            .rename(columns = {0:"sold_qty"})
df

,customer_id,article_id,sold_qty
0,37865,678342001,5
1,39574,507909001,4
2,26097,189634001,3
3,25061,684209012,3
4,65998,694966001,3
...,...,...,...
317136,62083,717664001,1
317137,62083,610776002,1
317138,62083,578476001,1
317139,62082,715343007,1


In [12]:
df.columns = ['user','item','rating']
df = df.iloc[:20000]
df

,user,item,rating
0,37865,678342001,5
1,39574,507909001,4
2,26097,189634001,3
3,25061,684209012,3
4,65998,694966001,3
...,...,...,...
19995,131736,757286005,1
19996,131726,833493002,1
19997,131736,751994009,1
19998,131736,705734001,1


### 7.2. Выполнить коллаборативную фильтрацию по схожести покупателей

In [13]:
from sklearn.neighbors import NearestNeighbors  
from collections import defaultdict
from scipy.sparse import csr_matrix 

df_matrix = df.pivot(index= 'user',columns='item',values='rating').fillna(0)
w1_pivot_matrix = csr_matrix(df_matrix)


knn = NearestNeighbors(n_neighbors=10, algorithm= 'brute', metric= 'cosine')
model_knn = knn.fit(w1_pivot_matrix)

def most_similar_users_to(user_id):
    most_similar_users_to = []
    distance, indice = model_knn.kneighbors(df_matrix.iloc[user_id,:].values.reshape(1,-1), n_neighbors=10)
    for i in range(1, len(distance.flatten())):
        user_id1 = df_matrix.index[indice.flatten()[i]]
        most_similar_users_to.append((user_id1, distance.flatten()[i]))

    most_similar_users_to.sort(key=lambda x: x[1], reverse=True)

    return most_similar_users_to[:10] 

In [14]:
def user_based_suggestions(user_id):
    # суммировать все коэффициенты подобия
    suggestions = defaultdict(float)
    non_interacted_items = df_matrix.iloc[user_id][df_matrix.iloc[user_id]==0].index.tolist()
    for other_user_id, similarity in most_similar_users_to(user_id):
        items_user_id = df_matrix.loc[other_user_id][df_matrix.loc[other_user_id]>0]
        for interest in items_user_id.index.tolist():
            if interest in non_interacted_items:
                 # для вывода списка товаров/фильмов,  df_items - датафрейм с названиями товаров/фильмов
                item_name = df_item[df_item['article_id']==interest]['product_name'].values[0]
                suggestions[item_name] += similarity
    # преобразовать их в сортированный список
    suggestions = sorted(suggestions.items(),
                         key=lambda x: x[1],
                         reverse=True)
    return suggestions[:10]   

### 7.3. Вывести в качестве рекомендаций список товаров для пользователя с ИД = 6510

In [15]:
print("Рекомендации для пользователя")
print(user_based_suggestions(6510))

Рекомендации для пользователя
[('T-shirt', 2.0), ('Underwear bottom', 2.0), ('Trousers', 1.0), ('Underwear Tights', 1.0), ('Dungarees', 1.0), ('Bikini top', 1.0), ('Dress', 1.0)]
